<h1><center><b><u>New York Taxis</u></b></center></h1>
<center><u><i>Anaysis of the NYC Taxis over the 2019-2020 Period</i></u></center>

## Set up Environment

### Add custom styling

Add some custom styling, to ensure that our document is consistently pretty.

In [18]:
%%html
<style>
    h1 {
        margin-top: 0px;
        margin-left: 0px;
        font-size: 40px;
        font-weight: bold;
        font-style: normal;    
    }
    h2 {
        margin-top: 60px;
        margin-left: 0px;
        font-size: 30px;
        font-weight: bold;
        font-style: normal;
    }
    h3 {
        margin-top: 40px;
        margin-left: 40px;
        font-size: 20px;
        font-weight: normal;
        font-style: normal;
    }
    h4 {
        margin-top: 20;
        margin-left: 80px;
        font-size: 15px;
        font-weight: normal;
        font-style: italic;
    }
</style>

### Load Global Packages

In [19]:
# Global imports
import os, sys, subprocess
from pprint import pprint
import pandas as pd
from pyspark.sql import SparkSession, functions as F
# from pyspark.sql.types import IntegerType, TimestampType, FloatType, LongType
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
# from pyspark.ml.regression import RandomForestRegressor

In [20]:
# Pandas options
pd.set_option("max_columns", None)
pd.set_option("max_rows", 100)

### Set the Working Directory

Set up the environment, and ensure that the current working directory (`os.getcwd()`) is always in the correct location. This is to fix a nuance in Jupyter that always sets the current working directory as the `__file__` location.

In [21]:
# Determine if running on docker or local
# envir = "local"
envir = "docker"

In [22]:
# Fix local issue
if envir == "local":

    # Ensure the directory is correct... every time.
    for i in range(5):
        if not os.getcwd().lower() == subprocess.run("git rev-parse --show-toplevel", stdout=subprocess.PIPE).stdout.decode("utf-8").replace("/","\\").strip().lower():
            os.chdir("..")
        else:
            break

    # Set up sys path environment
    if not os.path.abspath(".") in sys.path: sys.path.append(os.path.abspath("."))


In [23]:
# Fix docker issue
if envir == "docker":
    
    for i in range(5):
        if not os.getcwd().lower() == "mdsi_bde_aut21_at3":
            os.chdir("..")
        else:
            break
    if not os.path.abspath(".") in sys.path: sys.path.append(os.path.abspath("."))

In [24]:
# Autoreload
%reload_ext autoreload
%autoreload 2

### Load Local Packages

In [31]:
# Local imports
# from src.data import s3
# from src.pyspark import ps
from src.utils import assertions as a, misc as ms

## Set up Kafka

- URL: http://localhost:9021/

## Set up Spark

In [32]:
# Instantiate
spark = SparkSession \
    .builder \
    .appName("kafka") \
    .getOrCreate()

In [33]:
# Check
print(f"Spark: {spark.version}")
print(f"Hadoop: {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Spark: 3.1.1
Hadoop: 3.2.0


## Get Data

In [34]:
stream_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:29092") \
    .option("startingOffsets", "earliest") \
    .option("subscribe", "iex") \
    .load()

In [35]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [38]:
from src.producers import twitter_kafka_producer as tw
twitter_kafka_producer

<module 'src.producers.twitter_kafka_producer' from '/home/jovyan/work/src/producers/twitter_kafka_producer.py'>